In [11]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.2f}'.format)
all_data = pd.read_csv('/Users/bryan/Projects/finance-app/global-trends/backend/src/data/real_data.csv')
del all_data['Series Code']
del all_data['Country Code']
all_data.drop([32,33,34,35,36], inplace=True)
all_data.replace('..', np.nan, inplace=True)
for col in all_data.columns:
    if col not in ['Country Name', 'Series Name']:
        all_data[col] = pd.to_numeric(all_data[col], errors='coerce')

numeric_cols = all_data.drop(['Country Name', 'Series Name'], axis=1)
numeric_cols.interpolate(method='linear', axis=1, inplace=True, limit_direction='forward')
numeric_cols.bfill(axis=1, inplace=True)
mask = all_data.isna()
all_data[mask] = numeric_cols[mask]
replacements = {
    'GNI, Atlas method (current US$)': 'GNI',
    'GDP (current US$)': 'GDP',
    'Unemployment, total (% of total labor force) (national estimate)': 'Unemployment',
    'Inflation, consumer prices (annual %)': 'Inflation'
}

yearReplacements = {
    '2003 [YR2003]': 2003,
    '2004 [YR2004]': 2004,
    '2005 [YR2005]': 2005,
    '2006 [YR2006]': 2006,
    '2007 [YR2007]': 2007,
    '2008 [YR2008]': 2008,
    '2009 [YR2009]': 2009,
    '2010 [YR2010]': 2010,
    '2011 [YR2011]': 2011,
    '2012 [YR2012]': 2012,
    '2013 [YR2013]': 2013,
    '2014 [YR2014]': 2014,
    '2015 [YR2015]': 2015,
    '2016 [YR2016]': 2016,
    '2017 [YR2017]': 2017,
    '2018 [YR2018]': 2018,
    '2019 [YR2019]': 2019,
    '2020 [YR2020]': 2020,
    '2021 [YR2021]': 2021,
    '2022 [YR2022]': 2022,
}
all_data['Series Name'] = all_data['Series Name'].replace(replacements)
all_data.rename(columns=yearReplacements, inplace=True)
eap = all_data[all_data['Country Name'] == 'East Asia & Pacific']
del eap['Country Name']

eca = all_data[all_data['Country Name'] == 'Europe & Central Asia']
del eca['Country Name']

lac = all_data[all_data['Country Name'] == 'Latin America & Caribbean']
del lac['Country Name']

mena  = all_data[all_data['Country Name'] == 'Middle East & North Africa']
del mena['Country Name']

ssa = all_data[all_data['Country Name'] == 'Sub-Saharan Africa']
del ssa['Country Name']

world = all_data[all_data['Country Name'] == 'World']
del world['Country Name']

na = all_data[all_data['Country Name'] == 'North America']
del na['Country Name']

sa = all_data[all_data['Country Name'] == 'South Asia']
del sa['Country Name']

dataDict = {
    'world': world,
    'na': na,
    'eca': eca,
    'lac': lac,
    'ssa': ssa,
    'sa': sa,
    'eap': eap,
    'mena': mena
}
json_data = {}
for key,value in dataDict.items():
    json_data[key] = value.to_dict(orient='records')

all_data

,Country Name,Series Name,2003,2004,2005,2006,2007,2008,2009,2010,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,East Asia & Pacific,GNI,8594734187915.81,9739893268511.89,10745728012056.00,11436986685636.60,12310565407454.80,13503981910628.50,14358036862228.90,16261514589700.10,...,21973847629764.90,22642438410638.20,22530618959545.50,22863415146293.00,23807630755388.10,25984649921714.30,27520793520030.80,27518139023935.40,30396238074547.10,32060244329199.60
1,East Asia & Pacific,Unemployment,4.38,4.36,4.45,4.33,4.15,4.32,4.30,4.02,...,3.77,3.76,3.68,3.60,3.52,4.19,4.21,4.69,4.44,4.44
2,East Asia & Pacific,GDP,8702207399274.63,9758433473126.78,10411317309554.80,11034808224299.40,12331154735962.70,14220141864723.50,14630626964216.50,17078044883355.30,...,21411260037373.00,22089565874969.30,21995286337605.60,22775818467016.90,24329068203247.40,26487849774769.50,27032674446875.70,27148447173104.20,31112165534854.60,30688235798301.00
3,East Asia & Pacific,Inflation,2.29,3.24,3.04,3.94,3.76,8.34,1.62,3.06,...,2.58,2.51,0.80,1.28,1.80,2.31,1.70,1.28,2.41,5.41
4,Europe & Central Asia,GNI,11831857714938.30,14434171722826.50,16772106469828.70,18209008828855.60,19793052516290.10,21647794096346.80,21531111309986.60,21820863892390.50,...,23374933568546.40,23525170741530.60,22379996588641.70,21279596451467.30,20924909503963.90,22426491748503.30,23346747304522.90,22166768631993.30,24711982686822.20,26246382063331.80
5,Europe & Central Asia,Unemployment,8.69,8.69,8.35,7.91,7.17,7.15,9.07,9.15,...,9.21,8.89,8.51,8.09,7.48,6.98,6.74,7.28,7.12,7.12
6,Europe & Central Asia,GDP,13534592292253.00,15775499878104.70,16780902188907.00,18167734620079.10,21227695909963.30,23363317725408.30,20544984633327.10,21016244485056.30,...,23454504187753.00,23783815080046.50,20480970346558.50,20408422727022.50,21652739213015.30,23188460071899.60,22907909231787.30,22156584402676.90,25312104726364.40,25329956337827.00
7,Europe & Central Asia,Inflation,2.35,2.83,2.61,3.19,2.92,6.06,1.54,2.42,...,1.75,0.62,0.41,0.39,1.96,2.04,1.82,1.21,3.25,10.39
8,Latin America & Caribbean,GNI,2091471298691.86,2293592786248.08,2595428739728.45,3017745778918.46,3555697557927.79,4138242773489.45,4277205863715.31,4791609838773.43,...,6271532989335.27,6269862938383.71,5732492411457.21,5326157881370.71,5303656432246.48,5568174212312.89,5683933585085.25,5017542242903.42,5361604347678.60,5782664917202.84
9,Latin America & Caribbean,Unemployment,9.41,8.45,8.33,7.24,7.36,6.75,7.78,7.19,...,6.20,6.01,6.53,7.60,7.94,7.95,8.14,10.51,9.31,6.97
